Drive + Configurações Gerais

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np

BASE_IN = "/content/drive/MyDrive/Mestrado/Dados Gerais"
BASE_OUT = "/content/drive/MyDrive/Mestrado/Dados_Anonimizados"

ANOS = [2018, 2019, 2020, 2021, 2022, 2023]

LAT_COL = "latitude"
LON_COL = "longitude"

os.makedirs(BASE_OUT, exist_ok=True)

print("📂 Origem :", BASE_IN)
print("📂 Destino:", BASE_OUT)
print("📅 Anos   :", ANOS)


Mounted at /content/drive
📂 Origem : /content/drive/MyDrive/Mestrado/Dados Gerais
📂 Destino: /content/drive/MyDrive/Mestrado/Dados_Anonimizados
📅 Anos   : [2018, 2019, 2020, 2021, 2022, 2023]


scripts de anonimização

In [7]:
import numpy as np
import pandas as pd

def _planar_laplace_noise_vector(n, epsilon, sensitivity_meters, rng=None):
    """
    Gera n deslocamentos (dx, dy) em METROS aproximando o ruído Planar Laplace.
    Implementação prática para DP geográfica (geo-indistinguishability).

    Observação: Para artigo, o importante é manter o mecanismo consistente entre execuções
    e registrar epsilon e sensitivity.
    """
    if rng is None:
        rng = np.random.default_rng(42)

    # Ângulo uniforme
    theta = rng.uniform(0, 2*np.pi, size=n)

    # Raio: distribuição Gamma(k=2, scale=sensitivity/epsilon)
    # Isso gera uma distribuição radial compatível com o mecanismo planar Laplace.
    scale = sensitivity_meters / float(epsilon)
    r = rng.gamma(shape=2.0, scale=scale, size=n)

    dx = r * np.cos(theta)
    dy = r * np.sin(theta)
    return dx, dy

def add_planar_laplace_noise(df, epsilon=1.0, sensitivity_meters=1000.0,
                            lat_col="latitude", lon_col="longitude", seed=42):
    """
    Adiciona ruído DP em latitude/longitude.
    - epsilon: orçamento de privacidade
    - sensitivity_meters: sensibilidade (metros)
    - converte deslocamento em metros -> graus (aprox)
    """
    out = df.copy()
    rng = np.random.default_rng(seed)

    lat = pd.to_numeric(out[lat_col], errors="coerce")
    lon = pd.to_numeric(out[lon_col], errors="coerce")
    mask = lat.notna() & lon.notna()
    out = out.loc[mask].copy()

    lat_vals = out[lat_col].astype(float).values
    lon_vals = out[lon_col].astype(float).values

    n = len(out)
    dx_m, dy_m = _planar_laplace_noise_vector(
        n=n, epsilon=epsilon, sensitivity_meters=sensitivity_meters, rng=rng
    )

    # Conversão aproximada:
    # 1 grau latitude ~ 111_320 m
    # 1 grau longitude ~ 111_320 * cos(latitude) m
    meters_per_deg_lat = 111_320.0
    lat_rad = np.deg2rad(lat_vals)
    meters_per_deg_lon = 111_320.0 * np.cos(lat_rad)
    meters_per_deg_lon = np.where(meters_per_deg_lon == 0, 1e-9, meters_per_deg_lon)

    dlat = dy_m / meters_per_deg_lat
    dlon = dx_m / meters_per_deg_lon

    out[lat_col] = lat_vals + dlat
    out[lon_col] = lon_vals + dlon

    return out


In [3]:
def permutacao(df):
    out = df.copy()
    out[LAT_COL] = np.random.permutation(out[LAT_COL].values)
    out[LON_COL] = np.random.permutation(out[LON_COL].values)
    return out

def generalizacao(df, casas=1):
    out = df.copy()
    out[LAT_COL] = out[LAT_COL].round(casas)
    out[LON_COL] = out[LON_COL].round(casas)
    return out

def microagregacao(df, k=5):
    out = df.copy().sample(frac=1, random_state=42).reset_index(drop=True)
    for i in range(0, len(out), k):
        grupo = out.iloc[i:i+k]
        out.loc[i:i+k-1, LAT_COL] = grupo[LAT_COL].mean()
        out.loc[i:i+k-1, LON_COL] = grupo[LON_COL].mean()
    return out

def k_anonimato(df, k=2):
    return microagregacao(df, k=k)

def privacidade_diferencial(df, epsilon=1.0, sensitivity_meters=1000.0):
    out = df.copy()
    out[LAT_COL] = pd.to_numeric(out[LAT_COL], errors="coerce")
    out[LON_COL] = pd.to_numeric(out[LON_COL], errors="coerce")
    out = out.dropna(subset=[LAT_COL, LON_COL])

    # aplica DP
    out = add_planar_laplace_noise(
        out,
        epsilon=epsilon,
        sensitivity_meters=sensitivity_meters,
        lat_col=LAT_COL,
        lon_col=LON_COL,
        seed=42
    )
    return out



In [8]:
TECNICAS = [
    ("permutacao", lambda df: permutacao(df)),
    ("generalizacao_dec1", lambda df: generalizacao(df, casas=1)),
    ("microagregacao_k5", lambda df: microagregacao(df, k=5)),
    ("k_anon_k2", lambda df: k_anonimato(df, k=2)),
]

EPSILONS_DP = [1.0]  # baseline atual
SENSITIVITY_DP = 1000.0  # metros


In [9]:
for ano in ANOS:
    in_path = f"{BASE_IN}/{ano}_COMPLETO.csv"
    print(f"\n📥 Lendo {in_path}")

    df = pd.read_csv(in_path, sep=";", low_memory=False)

    df[LAT_COL] = pd.to_numeric(df[LAT_COL], errors="coerce")
    df[LON_COL] = pd.to_numeric(df[LON_COL], errors="coerce")
    df = df.dropna(subset=[LAT_COL, LON_COL])

    # Salvar versão "original padronizada"
    out_orig_dir = f"{BASE_OUT}/original"
    os.makedirs(out_orig_dir, exist_ok=True)
    df.to_csv(f"{out_orig_dir}/{ano}.csv", index=False)
    print("  ✅ original salvo")

    # Técnicas clássicas
    for nome, func in TECNICAS:
        out_dir = f"{BASE_OUT}/{nome}"
        os.makedirs(out_dir, exist_ok=True)

        df_anon = func(df)
        out_file = f"{out_dir}/{ano}.csv"
        df_anon.to_csv(out_file, index=False)

        print(f"  ✅ {nome} salvo")

    # Privacidade Diferencial
    for eps in EPSILONS_DP:
        out_dir = f"{BASE_OUT}/dp/eps_{eps}"
        os.makedirs(out_dir, exist_ok=True)

        df_dp = privacidade_diferencial(
            df,
            epsilon=eps,
            sensitivity_meters=SENSITIVITY_DP
        )
        out_file = f"{out_dir}/{ano}.csv"
        df_dp.to_csv(out_file, index=False)

        print(f"  ✅ dp (ε={eps}) salvo")



📥 Lendo /content/drive/MyDrive/Mestrado/Dados Gerais/2018_COMPLETO.csv
  ✅ original salvo
  ✅ permutacao salvo
  ✅ generalizacao_dec1 salvo
  ✅ microagregacao_k5 salvo
  ✅ k_anon_k2 salvo
  ✅ dp (ε=1.0) salvo

📥 Lendo /content/drive/MyDrive/Mestrado/Dados Gerais/2019_COMPLETO.csv
  ✅ original salvo
  ✅ permutacao salvo
  ✅ generalizacao_dec1 salvo
  ✅ microagregacao_k5 salvo
  ✅ k_anon_k2 salvo
  ✅ dp (ε=1.0) salvo

📥 Lendo /content/drive/MyDrive/Mestrado/Dados Gerais/2020_COMPLETO.csv
  ✅ original salvo
  ✅ permutacao salvo
  ✅ generalizacao_dec1 salvo
  ✅ microagregacao_k5 salvo
  ✅ k_anon_k2 salvo
  ✅ dp (ε=1.0) salvo

📥 Lendo /content/drive/MyDrive/Mestrado/Dados Gerais/2021_COMPLETO.csv
  ✅ original salvo
  ✅ permutacao salvo
  ✅ generalizacao_dec1 salvo
  ✅ microagregacao_k5 salvo
  ✅ k_anon_k2 salvo
  ✅ dp (ε=1.0) salvo

📥 Lendo /content/drive/MyDrive/Mestrado/Dados Gerais/2022_COMPLETO.csv
  ✅ original salvo
  ✅ permutacao salvo
  ✅ generalizacao_dec1 salvo
  ✅ microagregacao_